In [2]:
import os


# Lấy đường dẫn của thư mục hiện tại
current_dir_path = os.getcwd()

# Chuyển đổi \ thành /
current_dir_path = current_dir_path.replace("\\", "/")

# In ra đường dẫn đã chuyển đổi
print(f"Đường dẫn hiện tại của thư mục (Unix style): {current_dir_path}")


Đường dẫn hiện tại của thư mục (Unix style): C:/Users/USER/flowerai


In [4]:
import os

def print_directory_structure(root_dir, indent_level=0):
    # Tạo khoảng trắng để thụt dòng dựa trên mức độ indent
    indent = '    ' * indent_level
    print(f"{indent}{os.path.basename(root_dir)}/")

    # Duyệt qua tất cả các tệp và thư mục con
    items = os.listdir(root_dir)
    files_printed = 0
    for item in items:
        item_path = os.path.join(root_dir, item)
        
        if os.path.isdir(item_path):
            # Nếu là thư mục, tiếp tục đệ quy
            print_directory_structure(item_path, indent_level + 1)
        else:
            # In ra tối đa 2 tệp
            if files_printed < 2:
                print(f"{indent}    {item}")  # In ra tệp
                files_printed += 1
            elif files_printed == 2:
                print(f"{indent}    ... (còn nhiều tệp khác)")  # Hiển thị thông báo nếu còn nhiều tệp
                break

# Đường dẫn tới thư mục cần in ra cấu trúc
root_directory = current_dir_path

# In ra cấu trúc thư mục với tối đa 2 tệp trong mỗi thư mục
print_directory_structure(root_directory)


flowerai/
    .ipynb_checkpoints/
        colabnote-checkpoint.ipynb
        split_data-checkpoint.ipynb
        ... (còn nhiều tệp khác)
    colabnote.ipynb
    data.yaml
    dataset_flower.v4i.yolov8/
        README.dataset.txt
        README.roboflow.txt
        test/
            images/
                image_00266_jpg.rf.719ca327987f9186b5341b20da080ef6.jpg
                image_00276_jpg.rf.321fd3312f3823bc41187a8c603df211.jpg
                ... (còn nhiều tệp khác)
            labels/
                image_00266_jpg.rf.719ca327987f9186b5341b20da080ef6.txt
                image_00276_jpg.rf.321fd3312f3823bc41187a8c603df211.txt
                ... (còn nhiều tệp khác)
        train/
            images/
                image_00252_jpg.rf.5ed5c2aff5d22777b68611235357652a.jpg
                image_00252_jpg.rf.99327eb30968ffc343986bc5e38fa240.jpg
                ... (còn nhiều tệp khác)
            labels/
                image_00252_jpg.rf.5ed5c2aff5d22777b68611235357652a.txt
      

In [6]:
merged_dataset_path = os.path.join(root_directory, 'dataset_flower.v4i.yolov8')

# Sửa lại việc in đường dẫn
print(os.path.join(merged_dataset_path, 'train/images'))
print(os.path.join(merged_dataset_path, 'valid/images'))
print(os.path.join(merged_dataset_path, 'test/images'))
print(os.path.join(merged_dataset_path, 'data.yaml'))
print_directory_structure(merged_dataset_path)

C:/Users/USER/flowerai\dataset_flower.v4i.yolov8\train/images
C:/Users/USER/flowerai\dataset_flower.v4i.yolov8\valid/images
C:/Users/USER/flowerai\dataset_flower.v4i.yolov8\test/images
C:/Users/USER/flowerai\dataset_flower.v4i.yolov8\data.yaml
dataset_flower.v4i.yolov8/
    README.dataset.txt
    README.roboflow.txt
    test/
        images/
            image_00266_jpg.rf.719ca327987f9186b5341b20da080ef6.jpg
            image_00276_jpg.rf.321fd3312f3823bc41187a8c603df211.jpg
            ... (còn nhiều tệp khác)
        labels/
            image_00266_jpg.rf.719ca327987f9186b5341b20da080ef6.txt
            image_00276_jpg.rf.321fd3312f3823bc41187a8c603df211.txt
            ... (còn nhiều tệp khác)
    train/
        images/
            image_00252_jpg.rf.5ed5c2aff5d22777b68611235357652a.jpg
            image_00252_jpg.rf.99327eb30968ffc343986bc5e38fa240.jpg
            ... (còn nhiều tệp khác)
        labels/
            image_00252_jpg.rf.5ed5c2aff5d22777b68611235357652a.txt
          

In [38]:
import os
import shutil
import random
import yaml  # Thêm import để đọc tệp YAML

def copy_dataset_part(src_images_dir, src_labels_dir, dest_dir, part_num, total_parts):
    # Tạo thư mục cho phần dữ liệu
    part_images_dir = os.path.join(dest_dir, f'part_{part_num}/images')
    part_labels_dir = os.path.join(dest_dir, f'part_{part_num}/labels')
    
    # Kiểm tra xem thư mục đã tồn tại chưa
    if os.path.exists(part_images_dir) and os.path.exists(part_labels_dir):
        print(f"Thư mục cho phần {part_num} đã tồn tại. Bỏ qua việc sao chép.")
        return

    os.makedirs(part_images_dir, exist_ok=True)
    os.makedirs(part_labels_dir, exist_ok=True)

    # Lấy danh sách tất cả các tệp hình ảnh
    all_images = os.listdir(src_images_dir)
    random.shuffle(all_images)  # Trộn ngẫu nhiên danh sách hình ảnh

    # Tính kích thước phần
    part_size = len(all_images) // total_parts
    start_index = (part_num - 1) * part_size
    end_index = start_index + part_size if part_num < total_parts else len(all_images)

    # Sao chép hình ảnh và nhãn cho phần hiện tại
    for image_name in all_images[start_index:end_index]:
        src_image_path = os.path.join(src_images_dir, image_name)
        src_label_path = os.path.join(src_labels_dir, image_name.replace('.jpg', '.txt'))  # Thay đổi đuôi tệp cho nhãn

        # Sao chép tệp hình ảnh
        shutil.copy(src_image_path, os.path.join(part_images_dir, image_name))

        # Sao chép tệp nhãn
        if os.path.exists(src_label_path):
            shutil.copy(src_label_path, os.path.join(part_labels_dir, image_name.replace('.jpg', '.txt')))
        else:
            print(f"Không tìm thấy nhãn cho hình ảnh: {image_name}")

    print(f"Đã sao chép dữ liệu cho phần {part_num}.")

def create_yaml_file(part_num, output_dir, merged_data_file):
    # Đọc tên lớp từ tệp YAML chính
    with open(merged_data_file, 'r') as f:
        yaml_data = yaml.safe_load(f)

    class_names = yaml_data.get('names', [])  # Lấy danh sách tên lớp
    num_classes = len(class_names)

    # Đường dẫn tuyệt đối tới thư mục hình ảnh
    part_images_dir = os.path.join(output_dir, f'part_{part_num}/images')

    # Chuyển đổi đường dẫn tuyệt đối sang tương đối so với thư mục hiện tại
    rel_train_images_path = os.path.relpath(part_images_dir, output_dir)

    yaml_content = f"""
train: {rel_train_images_path}
val: {rel_train_images_path}
nc: {num_classes}
names: {class_names}
"""
    yaml_path = os.path.join(output_dir, f'part_{part_num}.yaml')
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(yaml_content.strip())

# Đường dẫn đến thư mục hình ảnh và nhãn
merged_dataset_path = os.path.join(root_directory, 'dataset_flower.v4i.yolov8')
train_images_dir = os.path.join(merged_dataset_path, 'train/images')
train_labels_dir = os.path.join(merged_dataset_path, 'train/labels')

# Thư mục đích để lưu các phần dữ liệu
output_dir = os.path.join(merged_dataset_path, 'split_dataset')
os.makedirs(output_dir, exist_ok=True)

# Đường dẫn đến tệp YAML chính (merged_data.yaml)
merged_data_file = os.path.join(merged_dataset_path, 'data.yaml')  # Đường dẫn đến tệp YAML

# Số phần muốn chia
num_parts = 35

# Sao chép dữ liệu cho từng phần
for i in range(1, num_parts + 1):
    copy_dataset_part(train_images_dir, train_labels_dir, output_dir, i, num_parts)
    create_yaml_file(i, output_dir, merged_data_file)  # Tạo tệp YAML cho từng phần

print(f"Đã hoàn tất chia và sao chép dữ liệu thành {num_parts} phần và tạo tệp YAML.")


Đã sao chép dữ liệu cho phần 1.
Đã sao chép dữ liệu cho phần 2.
Đã sao chép dữ liệu cho phần 3.
Đã sao chép dữ liệu cho phần 4.
Đã sao chép dữ liệu cho phần 5.
Đã sao chép dữ liệu cho phần 6.
Đã sao chép dữ liệu cho phần 7.
Đã sao chép dữ liệu cho phần 8.
Đã sao chép dữ liệu cho phần 9.
Đã sao chép dữ liệu cho phần 10.
Đã sao chép dữ liệu cho phần 11.
Đã sao chép dữ liệu cho phần 12.
Đã sao chép dữ liệu cho phần 13.
Đã sao chép dữ liệu cho phần 14.
Đã sao chép dữ liệu cho phần 15.
Đã sao chép dữ liệu cho phần 16.
Đã sao chép dữ liệu cho phần 17.
Đã sao chép dữ liệu cho phần 18.
Đã sao chép dữ liệu cho phần 19.
Đã sao chép dữ liệu cho phần 20.
Đã sao chép dữ liệu cho phần 21.
Đã sao chép dữ liệu cho phần 22.
Đã sao chép dữ liệu cho phần 23.
Đã sao chép dữ liệu cho phần 24.
Đã sao chép dữ liệu cho phần 25.
Đã sao chép dữ liệu cho phần 26.
Đã sao chép dữ liệu cho phần 27.
Đã sao chép dữ liệu cho phần 28.
Đã sao chép dữ liệu cho phần 29.
Đã sao chép dữ liệu cho phần 30.
Đã sao chép dữ liệu